In [3]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# from tensorflow.python.keras.datasets import imdb
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing import sequence


from tensorboard import summary as summary_lib

#전처리 lib
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.7.0


In [4]:
default_path='~/.kaggle/competitions/word2vec-nlp-tutorial/'
train = pd.read_csv(default_path+"labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

#매번 위와 같은 작업을 반복할 수 없으니 함수로 만들어봅니다.
def review_to_words( raw_review ):
    # 1. HTML 태그 지우기
    review_text = BeautifulSoup(raw_review, "html5lib").get_text()
    
    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들을 나누기
    words = letters_only.lower().split()

    # 4. 파이썬에서 리스트보다 set에서의 찾기가 더 빠르다고 합니다.
    stops = set(stopwords.words("english"))
    
    # 5. 불용어가 아닌 것만 남기기
    meaningful_words = [w for w in words if not w in stops]
    
    # 6. 불용어가 아닌 텍스트를 공백 단위로 나누어서 리턴
    return( " ".join( meaningful_words ))

#함수를 만들었으니 본격적으로 사용해봅시다.
#전체 데이터의 라인 수를 얻습니다.
num_reviews = train["review"].size

#보통 엄청 크고 아름다운 PC를 가졌다면 위처럼 실행하면 됩니다.
#아니라면....컴퓨터가 일을 제대로 하고 있는 지 확인이 필요할 겁니다.
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    #1000라인마다 잘하고 있는 지 보고서를 제출하게 합니다.
    if( (i+1)%1000 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [5]:
# tensorflow내의 내장된 keras기능을 사용하여, 문장을 전처리를 진행하고, 단어 vocab을 dictionary화 합니다.
t = Tokenizer()
t.fit_on_texts(clean_train_reviews)
variables = t.texts_to_sequences(clean_train_reviews)
labels = np.array(train['sentiment'])

from sklearn.model_selection import train_test_split

x_train_variable, x_test_variable, y_train, y_test = train_test_split(
            variables, labels, test_size=0.2, random_state=10)
            

vocab_size = 150000
sentence_size = 200
embedding_size = 50
model_dir = './checkpoint/cnn_classifier'

# vocabulary셋을 만들때 padding이나 start, unknown과 같은 토큰 index들을 설정을 해줘야 합니다.
# 여기의 경우는 padding은 start, unknown(oov)을 각각 0, 1, 2로 indexing해두었습니다.
pad_id = 0
start_id = 1
oov_id = 2
# 첫 시작 vocab은 3번부터 시작하기 위해 index_offset을 두어 원래 'vocabulary 토큰순서 + index offset'을 하도록 합니다.
# 예를 들어 'apple'이 첫 단어라고 하면 '0 + index_offset'한 위치가 vocabulary index 위치 입니다.
index_offset = 2

print("Pad sequences (samples x time)")
x_train = sequence.pad_sequences(x_train_variable, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test_variable, 
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post', 
                                value=pad_id)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

Pad sequences (samples x time)
x_train shape: (20000, 200)
x_test shape: (5000, 200)


In [6]:
# word indexing을 끝내고 vocabulary set을 생성합니다.
word_index = t.word_index
word_inverted_index = {v + index_offset: k for k, v in word_index.items()}

# 보통, 첫 index 위치에 padding과 start, unknown 토큰이 들어가도록 합니다.
word_inverted_index[pad_id] = '<PAD>'
word_inverted_index[start_id] = '<START>'
word_inverted_index[oov_id] = '<OOV>'

######################
# Test Vocabulary set#
######################
for i in range(0, 10):
    print(i, word_inverted_index[i])

def index_to_text(indexes):
    return ' '.join([word_inverted_index[i] for i in indexes])

print(index_to_text(variables[0]))

0 <PAD>
1 <START>
2 <OOV>
3 movie
4 film
5 one
6 like
7 good
8 time
9 even
stories look age korda happy lonely done back manages taking together fanshawe together knoxville dvd us well form jungle music enough would hilarious uwe dvd made less oscar aliens painful knoxville find testament find actual <OOV> tell look story title constant scary touching monkey korda fight c brosnan bad james giving redeeming much im context solid although wanted fashion begin nazi film korda type look feature makes women thought turned korda kazakos aymeric script <START> korda based time end first based maybe would idea working actual <OOV> whole dialogue car far kristel thrills carry local killing lordi tension mukhsin casting mysterious extreme close korda help much sequence korda halleck cartoons baloo killing lordi think journeymen turn get best frewer redeeming gave dyer dvd rick korda done surprised hilarious quite film korda industry kill circumstances least favor mexican local bad years right un

In [7]:
# 각 문장의 길이를 계산합니다. 이 경우엔 전처리 후에 문장 길이 정보를 보존하기 위해 사용합니다.
# 또한, 경우에 따라 네트워크에 padding정보를 포함하지 않기 위해 문장 길이정보를 필요로 할 때가 있습니다.
# 이 경우엔 문장길이는 최대 200으로 제한해두고 있습니다.
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

In [8]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [21]:
tfrecords_filename = 'bag_of_popcorns_indexing.tfrecords'

writer = tf.python_io.TFRecordWriter(tfrecords_filename)

# Let's collect the real images to later on compare
# to the reconstructed ones
original_tables = []

for review, label, length in zip(x_train, y_train, x_len_train):
    example = tf.train.Example(features=tf.train.Features(feature={
        'review': _bytes_feature(review.tostring()),
        'label': _int64_feature(label),
        'length': _int64_feature(length)}))
    
    writer.write(example.SerializeToString())

writer.close()

In [22]:
x_train[0]

array([ 1595,  5219,   312,  2004,  6412,    30,     3,    10,  3055,
        3718,   114,     2,   166,   209,  3034,    14,  5177,   237,
        6958,  1788, 19372,    17,  5219,     4,    30,    16,     4,
        2127,  7241,    33,    37,  1744,  2978,    43,   130,   433,
        3624, 13636,    16,   352,   327,    23,  2027,    36,  1686,
         141, 21432,    13,    59,     2,   222,    61,  4456,  1960,
          50,   671,   186,   854,  1653,  4071,    20,  7679,   649,
         743,   236,  8560,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,